### Ingest results.json file

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

##### Step 1 - Read the JOSN file uisng spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

In [0]:
results_schema = StructType(fields = [StructField("resultId", IntegerType(), False),
                                      StructField("raceId", IntegerType(), False),
                                      StructField("driverId", IntegerType(), False),
                                      StructField("constructorId", IntegerType(), False),
                                      StructField("number", IntegerType(), True),
                                      StructField("grid", IntegerType(), False),
                                      StructField("position", IntegerType(), True),
                                      StructField("positionText", StringType(), False),
                                      StructField("positionOrder", IntegerType(), False),
                                      StructField("points", FloatType(), False),
                                      StructField("laps", IntegerType(), False),
                                      StructField("time", StringType(), True),
                                      StructField("milliseconds", IntegerType(), True),
                                      StructField("fastestLap", IntegerType(), True),
                                      StructField("rank", IntegerType(), True),
                                      StructField("fastestLapTime", StringType(), True),
                                      StructField("fastestLapSpeed", StringType(), True),
                                      StructField("statusId", IntegerType(), False)
])

In [0]:
results_df = spark.read \
    .schema(results_schema) \
    .json(f"{raw_folder_path}/{v_file_date}/results.json")

In [0]:
# results_df.printSchema()

root
 |-- resultId: integer (nullable = true)
 |-- raceId: integer (nullable = true)
 |-- driverId: integer (nullable = true)
 |-- constructorId: integer (nullable = true)
 |-- number: integer (nullable = true)
 |-- grid: integer (nullable = true)
 |-- position: integer (nullable = true)
 |-- positionText: string (nullable = true)
 |-- positionOrder: integer (nullable = true)
 |-- points: float (nullable = true)
 |-- laps: integer (nullable = true)
 |-- time: string (nullable = true)
 |-- milliseconds: integer (nullable = true)
 |-- fastestLap: integer (nullable = true)
 |-- rank: integer (nullable = true)
 |-- fastestLapTime: string (nullable = true)
 |-- fastestLapSpeed: string (nullable = true)
 |-- statusId: integer (nullable = true)



In [0]:
# display(results_df.limit(5))

resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
24986,1053,830,9,33,3,1,1,1,25.0,63,2:02:34.598,7354598,60,2,1:17.524,227.96,1
24987,1053,1,131,44,1,2,2,2,19.0,63,+22.000,7376598,60,1,1:16.702,230.403,1
24988,1053,846,1,4,7,3,3,3,15.0,63,+23.702,7378300,63,3,1:18.259,225.819,1
24989,1053,844,6,16,4,4,4,4,12.0,63,+25.579,7380177,60,6,1:18.379,225.473,1
24990,1053,832,6,55,11,5,5,5,10.0,63,+27.036,7381634,60,7,1:18.490,225.154,1


##### Step 2 - Rename columns and add new columns

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
results_with_columns_df = add_ingestion_date(results_df) \
    .withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("resultId", "result_id") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("driverId", "driver_id") \
    .withColumnRenamed("constructorId", "constructor_id") \
    .withColumnRenamed("positionText", "position_text") \
    .withColumnRenamed("positionOrder", "position_order") \
    .withColumnRenamed("fastestLap", "fastest_lap") \
    .withColumnRenamed("fastestLapTime", "fastest_lap_time") \
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed") \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date))

##### Step 3 - Drop the unwanted columns

In [0]:
results_final_df = results_with_columns_df.drop(col("statusId"))

#### De-dupe the dataframe

- Here we are droping this because, in 2021-03-21 have duplicates data.

In [0]:
results_dedupped_df = results_final_df.dropDuplicates(['race_id', 'driver_id'])
# It deletes the duplicate data, who have same details 'race_id' and 'driver_id' more than 1.

##### Step 4 - Write the output to processed container

In [0]:
merge_condition = "tgt.result_id = src.result_id AND tgt.race_id = src.race_id"
merge_delta_deta(results_dedupped_df, "f1_processed", "results", processed_folder_path, merge_condition, "race_id")

In [0]:
# %sql
# SELECT race_id, driver_id, COUNT(1)
# FROM f1_processed.results
# GROUP BY race_id, driver_id
# HAVING COUNT(1)>1
# ORDER BY race_id, driver_id DESC 

race_id,driver_id,count(1)


In [0]:
# %sql
# SELECT race_id, COUNT(1)
# FROM f1_processed.results
# GROUP BY race_id
# ORDER BY race_id DESC
# LIMIT 5;

race_id,count(1)
1053,20
1052,20
1047,20
1046,20
1045,20


In [0]:
%sql
SELECT * FROM f1_processed.results
LIMIT 5;

result_id,race_id,driver_id,constructor_id,number,grid,position,position_text,position_order,points,laps,time,milliseconds,fastest_lap,rank,fastest_lap_time,fastest_lap_speed,ingestion_date,data_source,file_date
11944,496,105,25,4,22,11,11,11,0.0,59,\N,null,null,null,\N,\N,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
11946,496,110,1,8,13,null,R,13,0.0,51,\N,null,null,null,\N,\N,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
11947,496,117,4,15,4,null,R,14,0.0,48,\N,null,null,null,\N,\N,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
11962,496,118,53,36,0,null,F,29,0.0,0,\N,null,null,null,\N,\N,2023-12-22T10:51:17.188Z,Ergast,2021-03-21
11956,496,119,21,29,18,null,R,23,0.0,6,\N,null,null,null,\N,\N,2023-12-22T10:51:17.188Z,Ergast,2021-03-21


In [0]:
dbutils.notebook.exit("Success")